In [34]:
import re
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime


headers_websites = {"Referer":None,
                    "Sec-Ch-Ua":"'\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"'",
                    "Sec-Ch-Ua-Mobile":"?0",
                    "Sec-Ch-Ua-Platform": "Windows",
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
                   }


class main_email_scrapper:
    
    def __init__(self, original_url):
        self.original_url = original_url
        
        if self.original_url.upper().find('WWW.') < 0:
            self.original_url = self.original_url.replace('//','//www.') 
            
        self.start_time = datetime.datetime.now()
        self.end_time =  self.start_time + datetime.timedelta(seconds=5)
        # to save scraped urls
        self.scraped = set()
        self.steps = 0
        # to save fetched emails
        self.emails = set() 
        
        self.scrap_emails_from_url(original_url)
        
        
    def finished(self):
        return list(self.emails)
          
    def scrap_emails_from_url(self, url):
        if url.find('///') < 0 and url.find('tel:') < 0:
            if self.steps < 2:
                if datetime.datetime.now() >= self.end_time and len(list(self.emails)) < 1:
                    self.end_time =  self.end_time + datetime.timedelta(seconds=5)
                    self.steps += 1
                if datetime.datetime.now() <= self.end_time:


                    if url not in self.scraped:
                        self.scraped.add(url)

                        parts = urlsplit(url)

                        base_url = "{0.scheme}://{0.netloc}".format(parts)

                        if '/' in parts.path:
                            path = url[:url.rfind('/')+1]
                        else:
                            path = url

                        print(path)

                        print("Crawling URL %s" % url)
                        
                        headers_websites["Referer"] = url
                        
                        try:
                            r = requests.get(url, headers=headers_websites)
                            r.encoding = r.apparent_encoding
                        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.UnicodeDecodeError):
                            try:
                                r = requests.get(url, headers=headers_websites)
                                r.encoding = r.apparent_encoding
                            except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.UnicodeDecodeError):
                                pass

                                
                            
                        try:
                            new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-zA-Z]+", r.text if r.text else '', re.I))
                            new_emails_updated = set([new_email for new_email in list(new_emails) if new_email.upper().find('.JPG') < 0 and new_email.upper().find('.JPEG') < 0 and new_email.upper().find('.PNG') < 0 and new_email.upper().find('.WEBP') < 0 and new_email.upper().find('.MP4') < 0])
                            self.emails.update(new_emails_updated)
                        except UnboundLocalError:
                            pass
                     

                        soup = BeautifulSoup(r.text)
                        current_urls = set()
                        new_emails_set = set() 

                     
                        for anchor in soup.find_all("a"):

                            if "href" in anchor.attrs:
                                link = anchor.attrs["href"]
                            else:
                                link = ''

                            if (link.upper().find('.JPG') < 0 and link.upper().find('.JPEG') < 0 and link.upper().find('.PNG') < 0 and link.upper().find('.WEBP') < 0 and link.upper().find('.MP4') < 0) and link.find('javascript:') < 0:

                                if link.startswith('mailto:'):
                                    email = link.split(':')[1]
                                    self.emails.add(email)
                                else:
                                    print(link)
                                    if link.startswith('/') and not link.startswith('#'):
                                        url = path + link
                                    elif link.startswith(self.original_url) and link.upper().find('#') < 0:
                                        url = link
                                    
                                    current_urls.add(url)
                                    print(url)

                        current_urls_list = list(current_urls)
                        res = [i for i in current_urls_list if 'CONTACT' in i.upper()]
                        print(current_urls_list)
                        print(res)
                        if len(res) > 0:
                            for res_url in res:
                                self.scrap_emails_from_url(res_url)
                        for cur_url in current_urls_list:
                            self.scrap_emails_from_url(cur_url)
                                            


In [ ]:
email_scrapper = main_email_scrapper('https://archiconcept.fr/')
emails = email_scrapper.finished()


In [3]:
print(emails)

[]
